# Fine Tune for Credit Risk

In [ ]:
%pip install -r requirements.txt

In [1]:
import os
import openai
import json

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [11]:
import pandas as pd
import json

# read in csv file
df = pd.read_csv('credit_risk_dataset.csv')

# create list of dictionaries
dict_list = df.to_dict(orient='records')
# replace loan_status with 1 as Default or 0 as Paid
for dict in dict_list:
    if dict['loan_status'] == 1:
        dict['loan_status'] = "default"
    else:
        dict['loan_status'] = "paid"


[{'person_age': 22, 'person_income': 59000, 'person_home_ownership': 'RENT', 'person_emp_length': 123.0, 'loan_intent': 'PERSONAL', 'loan_grade': 'D', 'loan_amnt': 35000, 'loan_int_rate': 16.02, 'loan_status': 'default', 'loan_percent_income': 0.59, 'cb_person_default_on_file': 'Y', 'cb_person_cred_hist_length': 3}, {'person_age': 21, 'person_income': 9600, 'person_home_ownership': 'OWN', 'person_emp_length': 5.0, 'loan_intent': 'EDUCATION', 'loan_grade': 'B', 'loan_amnt': 1000, 'loan_int_rate': 11.14, 'loan_status': 'paid', 'loan_percent_income': 0.1, 'cb_person_default_on_file': 'N', 'cb_person_cred_hist_length': 2}, {'person_age': 25, 'person_income': 9600, 'person_home_ownership': 'MORTGAGE', 'person_emp_length': 1.0, 'loan_intent': 'MEDICAL', 'loan_grade': 'C', 'loan_amnt': 5500, 'loan_int_rate': 12.87, 'loan_status': 'default', 'loan_percent_income': 0.57, 'cb_person_default_on_file': 'N', 'cb_person_cred_hist_length': 3}, {'person_age': 23, 'person_income': 65500, 'person_home_o

In [12]:
from sklearn.model_selection import train_test_split
import json

train_data, validation_data = train_test_split(dict_list, test_size=0.2, random_state=42)


# define a function to write data to jsonl format
def write_jsonl(file_path, data):
    with open(file_path, 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')




# split the data into train and validation sets
train_data, validation_data = train_test_split(dict_list, test_size=0.2, random_state=42)

# remove loan_status from the train and validation sets

for dict in validation_data:
    dict.pop('loan_status')

# write train_data to jsonl format
write_jsonl('train_fdata.jsonl', train_data)

# write validation_data to jsonl format
write_jsonl('val_fdata.jsonl', validation_data)



KeyError: 'Name'